In [1]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Reading from CSV Files")
    .master("local[*]")
    .getOrCreate()
)

spark

In [8]:
df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("data/input/emp.csv") #spark identify the metadata so it creates a task

In [9]:
df.printSchema() 

root
 |-- employee_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- hire_date: timestamp (nullable = true)



In [10]:
df.show()

+-----------+-------------+-------------+---+------+------+-------------------+
|employee_id|department_id|         name|age|gender|salary|          hire_date|
+-----------+-------------+-------------+---+------+------+-------------------+
|          1|          101|     John Doe| 30|  Male| 50000|2015-01-01 00:00:00|
|          2|          101|   Jane Smith| 25|Female| 45000|2016-02-15 00:00:00|
|          3|          102|    Bob Brown| 35|  Male| 55000|2014-05-01 00:00:00|
|          4|          102|    Alice Lee| 28|Female| 48000|2017-09-30 00:00:00|
|          5|          103|    Jack Chan| 40|  Male| 60000|2013-04-01 00:00:00|
|          6|          103|    Jill Wong| 32|Female| 52000|2018-07-01 00:00:00|
|          7|          101|James Johnson| 42|  Male| 70000|2012-03-15 00:00:00|
|          8|          102|     Kate Kim| 29|Female| 51000|2019-10-01 00:00:00|
|          9|          103|      Tom Tan| 33|  Male| 58000|2016-06-01 00:00:00|
|         10|          104|     Lisa Lee

In [13]:
# Reading with Schema
_schema = "employee_id int, department_id int, name string, age int, gender string, salary double, hire_date date"

In [17]:
df_schema = spark.read.format("csv").option("header", True).schema(_schema).load("data/input/emp.csv")

In [18]:
df_schema.show()

+-----------+-------------+-------------+---+------+-------+----------+
|employee_id|department_id|         name|age|gender| salary| hire_date|
+-----------+-------------+-------------+---+------+-------+----------+
|          1|          101|     John Doe| 30|  Male|50000.0|2015-01-01|
|          2|          101|   Jane Smith| 25|Female|45000.0|2016-02-15|
|          3|          102|    Bob Brown| 35|  Male|55000.0|2014-05-01|
|          4|          102|    Alice Lee| 28|Female|48000.0|2017-09-30|
|          5|          103|    Jack Chan| 40|  Male|60000.0|2013-04-01|
|          6|          103|    Jill Wong| 32|Female|52000.0|2018-07-01|
|          7|          101|James Johnson| 42|  Male|70000.0|2012-03-15|
|          8|          102|     Kate Kim| 29|Female|51000.0|2019-10-01|
|          9|          103|      Tom Tan| 33|  Male|58000.0|2016-06-01|
|         10|          104|     Lisa Lee| 27|Female|47000.0|2018-08-01|
|         11|          104|   David Park| 38|  Male|65000.0|2015

In [33]:
# PERMISSIVE (default) # Ghi dữ liệu lỗi vào cột _corrupt_record, vẫn load các cột còn lại.
_schema = "id int, name string, age int, bad_record string"
df_p = spark.read.format("csv").schema(_schema).option("columnNameOfCorruptRecord", "bad_record").option("header", True).load("data/input/students_with_errors.csv")

In [30]:
df_p.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- bad_record: string (nullable = true)



In [32]:
df_p.where("bad_record is not null").show()

+----+-----+----+--------------+
|  id| name| age|    bad_record|
+----+-----+----+--------------+
|   4|David|null|4,David,twenty|
|null|Frank|  28|  six,Frank,28|
+----+-----+----+--------------+



In [54]:
# DROPMALFORMED Bỏ hẳn các dòng lỗi.

_schema = "id int, name string, age int"
df_m = spark.read.format("csv").option("header", True).option("mode", "DROPMALFORMED").schema(_schema).load("data/input/students_with_errors.csv")

In [55]:
df_m.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [56]:
df_m.show()

+---+-------+----+
| id|   name| age|
+---+-------+----+
|  1|  Alice|  23|
|  2|    Bob|  30|
|  3|Charlie|  25|
|  5|    Eve|  27|
|  7|  Grace|null|
+---+-------+----+



In [51]:
# FAILFAST Nếu gặp lỗi → dừng ngay (exception).

_schema = "id int, name string, age int"

df_m = (
    spark.read.format("csv")
    .option("header", True)
    .option("mode", "FAILFAST")   # đúng cú pháp
    .schema(_schema)              # schema phải gọi riêng
    .load("data/input/students_with_errors.csv")
)

In [52]:
df_m.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [53]:
df_m.show()

Py4JJavaError: An error occurred while calling o506.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 26) (e0295f6e0b86 executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1399)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:68)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:421)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apac

In [57]:
_option = {
    "header" : "true",
    "inferSchema" : "true",
    "mode" : "PERMISSIVE"
}

df = (spark.read.format("csv").options(**_option).load("data/input/emp.csv"))

In [58]:
df.show()

+-----------+-------------+-------------+---+------+------+-------------------+
|employee_id|department_id|         name|age|gender|salary|          hire_date|
+-----------+-------------+-------------+---+------+------+-------------------+
|          1|          101|     John Doe| 30|  Male| 50000|2015-01-01 00:00:00|
|          2|          101|   Jane Smith| 25|Female| 45000|2016-02-15 00:00:00|
|          3|          102|    Bob Brown| 35|  Male| 55000|2014-05-01 00:00:00|
|          4|          102|    Alice Lee| 28|Female| 48000|2017-09-30 00:00:00|
|          5|          103|    Jack Chan| 40|  Male| 60000|2013-04-01 00:00:00|
|          6|          103|    Jill Wong| 32|Female| 52000|2018-07-01 00:00:00|
|          7|          101|James Johnson| 42|  Male| 70000|2012-03-15 00:00:00|
|          8|          102|     Kate Kim| 29|Female| 51000|2019-10-01 00:00:00|
|          9|          103|      Tom Tan| 33|  Male| 58000|2016-06-01 00:00:00|
|         10|          104|     Lisa Lee